#### **Library imports**

In [1]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

#### **Hyperparams and loading data**

In [2]:
train_edges = np.load('data/train_edges.npy')
users = torch.LongTensor(train_edges[:, 0])
items = torch.LongTensor(train_edges[:, 1])
ratings = torch.FloatTensor(train_edges[:, 2])

n_users = 943 
n_items = 1682
n_samples = len(ratings)

#### **Defining collaborative filtering**

In [3]:
class CollaborativeFiltering(Module):
    def __init__(self, n_users, n_items, n_factors):
        super(CollaborativeFiltering, self).__init__()
        self.user_emb = nn.Embedding(n_users, n_factors)
        self.item_emb = nn.Embedding(n_items, n_factors)

    def forward(self, user, item):
        u = self.user_emb(user)
        i = self.item_emb(item)
        dot = (u * i).sum(1)
        return torch.sigmoid(dot)

#### **Code for inner loop of meta attack**

In [4]:
# makes code reproducible
torch.manual_seed(0)

# reload the users, items and ratings tensors
users = torch.LongTensor(train_edges[:, 0])
items = torch.LongTensor(train_edges[:, 1])
ratings = torch.FloatTensor(train_edges[:, 2])
ratings.requires_grad_() # set requires_grad = True for ratings
print('ratings: ', ratings)

lr = 10
T = 20

# define model and loss function
model = CollaborativeFiltering(n_users, n_items, n_factors = 64)
p1, p2 = model.parameters()
model.train()
loss_fn = nn.BCELoss(reduction = 'mean')

# for i in tqdm(range(T)):
for i in range(T):
    y_hat = model(users, items)
    loss = loss_fn(y_hat, ratings)
    print('inner loss at iter {}: {}'.format(i, loss.item()))
    
    p1_grad = torch.autograd.grad(loss, p1, create_graph=True)
    p2_grad = torch.autograd.grad(loss, p2, create_graph=True)

    p1_new = p1 - lr * p1_grad[0]
    p2_new = p2 - lr * p2_grad[0]

    with torch.no_grad():
        p1.copy_(p1_new)
        p2.copy_(p2_new)

meta_grad = torch.autograd.grad(loss, ratings)[0]
print('meta gradients: ', meta_grad)

ratings:  tensor([1., 1., 0.,  ..., 1., 1., 0.], requires_grad=True)
inner loss at iter 0: 4.066596031188965
inner loss at iter 1: 4.052426815032959
inner loss at iter 2: 4.040399551391602
inner loss at iter 3: 4.031717300415039
inner loss at iter 4: 4.02053165435791
inner loss at iter 5: 4.008946418762207
inner loss at iter 6: 3.9973649978637695
inner loss at iter 7: 3.9875125885009766
inner loss at iter 8: 3.975945234298706
inner loss at iter 9: 3.963573694229126
inner loss at iter 10: 3.9537487030029297
inner loss at iter 11: 3.9422755241394043
inner loss at iter 12: 3.9307312965393066
inner loss at iter 13: 3.9171624183654785
inner loss at iter 14: 3.908663034439087
inner loss at iter 15: 3.899308443069458
inner loss at iter 16: 3.887892484664917
inner loss at iter 17: 3.878131151199341
inner loss at iter 18: 3.8646483421325684
inner loss at iter 19: 3.854921340942383
meta gradients:  tensor([ 5.1595e-06,  2.0629e-05, -9.9811e-06,  ..., -1.5046e-05,
        -2.3528e-05,  1.8430e-05

#### **Select edge to perturb (add)**

In [21]:
max_meta_grad = -math.inf 
edge_to_add = -1
for i in range(n_samples):
    if ratings[i] == 0:
        if meta_grad[i] > max_meta_grad:
            max_meta_grad = meta_grad[i]
            edge_to_add = i 
print(max_meta_grad)
print(edge_to_add)
print(users[i], items[i], ratings[i])

tensor(0.0002)
141472
tensor(13) tensor(1628) tensor(0., grad_fn=<SelectBackward0>)


#### **Train inner loop for modified ratings**

In [34]:
ratings_mod = train_edges[:, 2].copy()
ratings_mod[edge_to_add] = 1
ratings_mod = torch.FloatTensor(ratings_mod)
ratings_mod.requires_grad_()

# set seed to make results reproducible
torch.manual_seed(0)

# define model and loss function
model = CollaborativeFiltering(n_users, n_items, n_factors = 64)
p1, p2 = model.parameters()
model.train()
loss_fn = nn.BCELoss(reduction = 'mean')

# for i in tqdm(range(T)):
T = 20
for i in range(T):
    y_hat = model(users, items)
    loss = loss_fn(y_hat, ratings_mod)
    print('inner loss at iter {}: {}'.format(i, loss.item()))
    
    p1_grad = torch.autograd.grad(loss, p1, create_graph=True)
    p2_grad = torch.autograd.grad(loss, p2, create_graph=True)

    p1_new = p1 - lr * p1_grad[0]
    p2_new = p2 - lr * p2_grad[0]

    with torch.no_grad():
        p1.copy_(p1_new)
        p2.copy_(p2_new)

meta_grad = torch.autograd.grad(loss, ratings_mod)[0]
print('meta gradients: ', meta_grad)

inner loss at iter 0: 4.066789150238037
inner loss at iter 1: 4.052619934082031
inner loss at iter 2: 4.040592670440674
inner loss at iter 3: 4.031909465789795
inner loss at iter 4: 4.020724296569824
inner loss at iter 5: 4.009138584136963
inner loss at iter 6: 3.9975569248199463
inner loss at iter 7: 3.987704277038574
inner loss at iter 8: 3.9761369228363037
inner loss at iter 9: 3.9637649059295654
inner loss at iter 10: 3.953939914703369
inner loss at iter 11: 3.9424662590026855
inner loss at iter 12: 3.930921792984009
inner loss at iter 13: 3.9173531532287598
inner loss at iter 14: 3.908853530883789
inner loss at iter 15: 3.899498462677002
inner loss at iter 16: 3.888082265853882
inner loss at iter 17: 3.8783206939697266
inner loss at iter 18: 3.864837884902954
inner loss at iter 19: 3.8551104068756104
meta gradients:  tensor([ 5.1595e-06,  2.0629e-05, -9.9811e-06,  ..., -1.5046e-05,
        -2.3528e-05,  1.8430e-05])


#### **Putting it together: code for outer loop of meta attack**

In [5]:
# some hyperparams
lr = 50
T = 10
Delta = 10
n_factors = 64

print('hyperparams used --')
print('learning rate: ', lr)
print('inner loop count T: ', T)
print('outer loop count Delta: ', Delta)
print('embedding size: ', n_factors)

# list of perturbations
edges_to_add = []

for delta in range(Delta):
    # reload the users, items and ratings tensors
    users = torch.LongTensor(train_edges[:, 0])
    items = torch.LongTensor(train_edges[:, 1])
    ratings = torch.FloatTensor(train_edges[:, 2])

    # add those perturbations to "ratings"
    for index in edges_to_add:
        ratings[index] = 1
    print('{} edges perturbed in "rating"'.format(len(edges_to_add)))

    # set requires_grad for ratings, to compute meta gradients
    ratings.requires_grad_()

    # makes code reproducible
    torch.manual_seed(0)

    # define model and loss
    model = CollaborativeFiltering(n_users, n_items, n_factors)
    p1, p2 = model.parameters()
    loss_fn = nn.BCELoss(reduction = 'mean')
    model.train()

    print('perturbation #', delta)
    # inner loop training process
    for i in range(T):
        y_hat = model(users, items)
        loss = loss_fn(y_hat, ratings)
        # if i % 5 == 0:
        print('inner loss at iter {}: {}'.format(i, loss.item()))
        
        p1_grad = torch.autograd.grad(loss, p1, create_graph=True)
        p2_grad = torch.autograd.grad(loss, p2, create_graph=True)

        p1_new = p1 - lr * p1_grad[0]
        p2_new = p2 - lr * p2_grad[0]

        with torch.no_grad():
            p1.copy_(p1_new)
            p2.copy_(p2_new)
    
    # compute meta gradient
    meta_grad = torch.autograd.grad(loss, ratings)[0]
    print('meta gradients: ', meta_grad)

    # select best edge to perturb
    max_meta_grad = -math.inf
    edge_to_add = -1
    search_space = 0
    for i in range(n_samples):
        if ratings[i] == 0: # search over only negative edges
            if meta_grad[i] > max_meta_grad:
                max_meta_grad = meta_grad[i]
                edge_to_add = i 
            search_space += 1
    print('edges searched: {}'.format(search_space))
    print('max meta gradient: {}, edge to add: {}'.format(max_meta_grad, edge_to_add))
    edges_to_add.append(edge_to_add)

    print("\n\n")

hyperparams used --
learning rate:  50
inner loop count T:  10
outer loop count Delta:  10
embedding size:  64
0 edges perturbed in "rating"
perturbation # 0
inner loss at iter 0: 4.066596031188965
inner loss at iter 1: 4.008872985839844
inner loss at iter 2: 3.953612804412842
inner loss at iter 3: 3.8982605934143066
inner loss at iter 4: 3.842863082885742
inner loss at iter 5: 3.7885544300079346
inner loss at iter 6: 3.735943555831909
inner loss at iter 7: 3.6827290058135986
inner loss at iter 8: 3.6294374465942383
inner loss at iter 9: 3.579831838607788
meta gradients:  tensor([ 3.9364e-06,  1.9463e-05, -8.3421e-06,  ..., -1.4434e-05,
        -2.3683e-05,  1.8197e-05])
edges searched: 100000
max meta gradient: 0.00018337038636673242, edge to add: 141472



1 edges perturbed in "rating"
perturbation # 1
inner loss at iter 0: 4.066789150238037
inner loss at iter 1: 4.0090651512146
inner loss at iter 2: 3.953803777694702
inner loss at iter 3: 3.8984506130218506
inner loss at iter 4: 3.8

In [35]:
model.parameters()

<generator object Module.parameters at 0x7f08714d2ad0>

In [37]:
ratings = torch.FloatTensor(train_edges[:, 2])
ratings[2] = 1
ratings

tensor([1., 1., 1.,  ..., 1., 1., 0.])